In [1]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
import math
import warnings
warnings.filterwarnings(action='ignore')
#import descartes
import folium
from tqdm.notebook import tqdm

In [2]:
%cd E:\yujin\INHA\data

E:\yujin\INHA\data


주차장 + 도시계획교통시설_주차장 + 야영장 + 생활체육시설 + 공공문화 체육시설 + 현재 설치되어 있는 충전기

# 1) 원래 데이터에 category넣고 테이블 통째로 합치기

In [102]:
gyboundary = gpd.read_file('20.광양시_행정경계(읍면동).geojson')

parkingLot_raw = pd.read_csv('02.광양시_주차장_공간정보.csv')
traf_parkingLot_raw = gpd.read_file('26.광양시_도시계획(교통시설).geojson')
campsite = pd.read_csv('05.광양시_대중집합시설_야영장.csv')
lifePys_raw = pd.read_csv('04.광양시_대중집합시설_생활체육시설.csv')
CulPys_raw = gpd.read_file('25.광양시_도시계획(공공문화체육시설).geojson')
chargingStation_raw = pd.read_csv('01.광양시_충전기설치현황.csv')

In [103]:
parkingLot_raw['category'] = '주차장'
parkingLot_raw.rename(columns={"면적":"DGM_AR"}, inplace=True)
traf_parkingLot_raw['category'] = '주차장'
campsite['category'] = '야영장'
lifePys_raw['category'] = '생활체육시설'
CulPys_raw['category'] = '공공문화체육시설'
chargingStation_raw['category'] = '충전기'

In [160]:
table = pd.concat([parkingLot_raw, traf_parkingLot_raw, campsite, lifePys_raw, CulPys_raw, chargingStation_raw], ignore_index=True)
table

,주차장명칭,구분,DGM_AR,주차면수,유료/무료,주소,lon,lat,category,PRESENT_SN,...,업종,상호,충전소명,충전소위치,충전기 운영기관,급속/완속,충전기용량,이용대수,충전기타입,요금정보(원/kw)
0,인동숲앞 주차장,노외,4189.0,161.0,유료,인서리 237-1,127.586883,34.970324,주차장,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,구터미널주차장,노외,2968.0,125.0,유료,중동 1651,127.695428,34.936252,주차장,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,구경찰서 주차장,노외,2836.0,89.0,무료,읍내리 227-1,127.584586,34.975637,주차장,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,칠성1지구,노외,1980.0,52.0,무료,칠성리 962-1,127.580683,34.970130,주차장,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,칠성1지구,노외,1970.0,64.0,무료,칠성리 948-1,127.580707,34.972523,주차장,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,NaN,NaN,NaN,NaN,NaN,NaN,127.612698,34.970095,충전기,NaN,...,NaN,NaN,송보파인빌 7차 아파트,전남 광양시 광양읍 와룡길 33 705동 707동 사이 주차장,한국전력,급속,NaN,1.0,DC차데모/AC3상/DC콤보,191.73
794,NaN,NaN,NaN,NaN,NaN,NaN,127.612698,34.970095,충전기,NaN,...,NaN,NaN,송보파인빌 7차 아파트,전남 광양시 광양읍 와룡길 33 705동 707동 사이 주차장,한국전력,완속,7kw/h,2.0,AC완속,191.73
795,NaN,NaN,NaN,NaN,NaN,NaN,127.610457,34.968399,충전기,NaN,...,NaN,NaN,남해오네뜨 아파트,전남 광양시 광양읍 와룡길 43 실내지하 109동 지하주차장 c15,한국전력,급속,NaN,1.0,DC차데모/AC3상/DC콤보,191.73
796,NaN,NaN,NaN,NaN,NaN,NaN,127.610457,34.968399,충전기,NaN,...,NaN,NaN,남해오네뜨 아파트,전남 광양시 광양읍 와룡길 43 실내지하 109동 지하주차장 c15,한국전력,완속,7kw/h,3.0,AC완속,191.73


# 2) 원래 테이블별 해야할 전처리 정리

1. 주차장  
1) 위도, 경도 합쳐서 geometry추가하기  
2) administration 추가하기  
3) 컬럼이름 변경 (DGM_AR -> area)  
4) 삭제할 컬럼 : 주차장명칭, 구분, 유료/무료, 주소  
     
     
2. 교통시설_주차장  
1) geometry에서(multipolygon) 위도, 경도 따와서 컬럼 새로 만들기  
2) administration 추가하기  
3) 컬럼이름 변경 (DGM_AR -> area)  
4) 삭제할컬럼 : PRESENT_SN, DGM_NM, DGM_LT  
  
  
3. 야영장  
1) 위도, 경도 합쳐서 geometry추가하기  
2) administration 추가하기  
3) 소유지 데이터에서 검색해서 면적컬럼 추가하기  
4) 삭제할컬럼 : 명칭, 주소  
  
  
4. 생활체육시설  
1) 위도, 경도 합쳐서 geometry추가하기  
2) administration 추가하기  
3) 소유지 데이터에서 검색해서 면적컬럼 추가하기  
4) 삭제할컬럼 : 업종, 상호, 주소  
  
  
5. 공공문화체육시설  
1) geometry에서(multipolygon) 위도, 경도 따와서 컬럼 새로 만들기    
2) administration 추가하기  
3) 소유지 데이터에서 검색해서 면적컬럼 추가하기  
4) 컬럼이름 변경 (DGM_AR -> area) 
5) 삭제할 컬럼 : PRESENT_SN, DGM_NM, DGM_LT  


6. 충전기   
1) 위도, 경도 합쳐서 geometry추가하기  
2) administration 추가하기  
3) 급속/완속 --> binary형태로 변경   
4) 삭제할 컬럼 : 충전소명, 충전소위치, 충전기 운영기관, 충전기용량, 이용대수, 충전기타입, 요금정보(원/kw)

# 3) 필요없는 컬럼 삭제

삭제할 컬럼 : ['주차장명칭', '구분', '유료/무료', '주소', 'PRESENT_SN', 'DGM_NM', 'DGM_LT','명칭', '업종','상호','충전소명', '충전소위치', 충전기 운영기관', '충전기용량', '이용대수', '충전기타입','요금정보(원/kw)']

In [161]:
table.drop(['주차장명칭', '구분', '유료/무료', '주소', 'PRESENT_SN', 'DGM_NM', 'DGM_LT','명칭', '업종','상호','충전소명', '충전소위치', '충전기 운영기관', '충전기용량', '이용대수', '충전기타입','요금정보(원/kw)'], axis=1, inplace=True)  #필요없는 열들 제거
table

,DGM_AR,주차면수,lon,lat,category,geometry,소유주체(공공/민간),급속/완속
0,4189.0,161.0,127.586883,34.970324,주차장,None,NaN,NaN
1,2968.0,125.0,127.695428,34.936252,주차장,None,NaN,NaN
2,2836.0,89.0,127.584586,34.975637,주차장,None,NaN,NaN
3,1980.0,52.0,127.580683,34.970130,주차장,None,NaN,NaN
4,1970.0,64.0,127.580707,34.972523,주차장,None,NaN,NaN
...,...,...,...,...,...,...,...,...
793,NaN,NaN,127.612698,34.970095,충전기,None,NaN,급속
794,NaN,NaN,127.612698,34.970095,충전기,None,NaN,완속
795,NaN,NaN,127.610457,34.968399,충전기,None,NaN,급속
796,NaN,NaN,127.610457,34.968399,충전기,None,NaN,완속


# 4) 전처리

## 4-1) lon,lat 값으로 geometry Point로 채우기.
## lon, lat가 nan이면 geomety에서 추출해서 채우기

In [162]:
#lon, lat가 nan이면 geometry에서 추출해서 채우기
for i in range(len(table)):
    if math.isnan(table['lon'][i]):
        table['lon'][i] = table['geometry'][i].centroid.x
        table['lat'][i] = table['geometry'][i].centroid.y

In [163]:
#lon, lat값으로 geometry Point로 채우기
table['geometry'] = table.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)    

## 4-2) 컬럼명 변경 : DGM_AR --> area

In [164]:
table.rename(columns={"DGM_AR":"area"}, inplace=True)
table

,area,주차면수,lon,lat,category,geometry,소유주체(공공/민간),급속/완속
0,4189.0,161.0,127.586883,34.970324,주차장,POINT (127.5868828 34.97032367),NaN,NaN
1,2968.0,125.0,127.695428,34.936252,주차장,POINT (127.6954279 34.93625189999999),NaN,NaN
2,2836.0,89.0,127.584586,34.975637,주차장,POINT (127.5845858 34.97563677),NaN,NaN
3,1980.0,52.0,127.580683,34.970130,주차장,POINT (127.5806828 34.97013011),NaN,NaN
4,1970.0,64.0,127.580707,34.972523,주차장,POINT (127.5807067 34.97252282),NaN,NaN
...,...,...,...,...,...,...,...,...
793,NaN,NaN,127.612698,34.970095,충전기,POINT (127.6126981 34.97009451),NaN,급속
794,NaN,NaN,127.612698,34.970095,충전기,POINT (127.6126981 34.97009451),NaN,완속
795,NaN,NaN,127.610457,34.968399,충전기,POINT (127.6104569 34.96839912),NaN,급속
796,NaN,NaN,127.610457,34.968399,충전기,POINT (127.6104569 34.96839912),NaN,완속


## 4-3) administration 컬럼 추가하기

In [174]:
table['administration']=np.nan
for i in range((len(table))):
    for j in range(len(gyboundary)):
        if table.geometry[i].within(gyboundary.geometry[j]) ==True:
            table['administration'][i] = gyboundary.ADM_DR_NM[j]

In [176]:
table

,area,주차면수,lon,lat,category,geometry,소유주체(공공/민간),급속/완속,administration
0,4189.0,161.0,127.586883,34.970324,주차장,POINT (127.5868828 34.97032367),NaN,NaN,광양읍
1,2968.0,125.0,127.695428,34.936252,주차장,POINT (127.6954279 34.93625189999999),NaN,NaN,중마동
2,2836.0,89.0,127.584586,34.975637,주차장,POINT (127.5845858 34.97563677),NaN,NaN,광양읍
3,1980.0,52.0,127.580683,34.970130,주차장,POINT (127.5806828 34.97013011),NaN,NaN,광양읍
4,1970.0,64.0,127.580707,34.972523,주차장,POINT (127.5807067 34.97252282),NaN,NaN,광양읍
...,...,...,...,...,...,...,...,...,...
793,NaN,NaN,127.612698,34.970095,충전기,POINT (127.6126981 34.97009451),NaN,급속,광양읍
794,NaN,NaN,127.612698,34.970095,충전기,POINT (127.6126981 34.97009451),NaN,완속,광양읍
795,NaN,NaN,127.610457,34.968399,충전기,POINT (127.6104569 34.96839912),NaN,급속,광양읍
796,NaN,NaN,127.610457,34.968399,충전기,POINT (127.6104569 34.96839912),NaN,완속,광양읍


In [184]:
not_in_Gwangyang = table[table['administration'].isnull()]
not_in_Gwangyang

,area,주차면수,lon,lat,category,geometry,소유주체(공공/민간),급속/완속,administration
362,14117.0,NaN,127.672218,34.929638,주차장,POINT (127.672218417643 34.92963843737846),NaN,NaN,NaN
365,342181.0,NaN,127.634632,34.905055,주차장,POINT (127.6346322090547 34.90505484246383),NaN,NaN,NaN
402,3440.0,NaN,127.622275,34.904543,주차장,POINT (127.6222750321843 34.90454265846324),NaN,NaN,NaN
403,2669.0,NaN,127.613386,34.914953,주차장,POINT (127.6133864445488 34.91495255129337),NaN,NaN,NaN
404,2032.0,NaN,127.612267,34.916964,주차장,POINT (127.6122665085071 34.91696351517145),NaN,NaN,NaN
405,4157.0,NaN,127.612039,34.918663,주차장,POINT (127.6120394985495 34.91866285270822),NaN,NaN,NaN
617,NaN,NaN,123.774740,32.531290,생활체육시설,POINT (123.77474 32.53129000000001),NaN,NaN,NaN


In [183]:
lat = 34.940613
lon = 127.695931
center = (lat, lon)

m = folium.Map(center, zoom_start=11)
folium.GeoJson(gyboundary).add_to(m)

for i in not_in_Gwangyang.index:
    sub_lat = not_in_Gwangyang.loc[i,"lat"]
    sub_lon = not_in_Gwangyang.loc[i,'lon']
    folium.Marker([sub_lat, sub_lon]).add_to(m)
m


In [186]:
not_in_Gwangyang.index.tolist()

[362, 365, 402, 403, 404, 405, 617]

In [187]:
table.drop(table.index[not_in_Gwangyang.index.tolist()], inplace=True)

,area,주차면수,lon,lat,category,geometry,소유주체(공공/민간),급속/완속,administration
0,4189.0,161.0,127.586883,34.970324,주차장,POINT (127.5868828 34.97032367),NaN,NaN,광양읍
1,2968.0,125.0,127.695428,34.936252,주차장,POINT (127.6954279 34.93625189999999),NaN,NaN,중마동
2,2836.0,89.0,127.584586,34.975637,주차장,POINT (127.5845858 34.97563677),NaN,NaN,광양읍
3,1980.0,52.0,127.580683,34.970130,주차장,POINT (127.5806828 34.97013011),NaN,NaN,광양읍
4,1970.0,64.0,127.580707,34.972523,주차장,POINT (127.5807067 34.97252282),NaN,NaN,광양읍
...,...,...,...,...,...,...,...,...,...
793,NaN,NaN,127.612698,34.970095,충전기,POINT (127.6126981 34.97009451),NaN,급속,광양읍
794,NaN,NaN,127.612698,34.970095,충전기,POINT (127.6126981 34.97009451),NaN,완속,광양읍
795,NaN,NaN,127.610457,34.968399,충전기,POINT (127.6104569 34.96839912),NaN,급속,광양읍
796,NaN,NaN,127.610457,34.968399,충전기,POINT (127.6104569 34.96839912),NaN,완속,광양읍


In [188]:
table.reset_index(drop=True,inplace=True)
table

,area,주차면수,lon,lat,category,geometry,소유주체(공공/민간),급속/완속,administration
0,4189.0,161.0,127.586883,34.970324,주차장,POINT (127.5868828 34.97032367),NaN,NaN,광양읍
1,2968.0,125.0,127.695428,34.936252,주차장,POINT (127.6954279 34.93625189999999),NaN,NaN,중마동
2,2836.0,89.0,127.584586,34.975637,주차장,POINT (127.5845858 34.97563677),NaN,NaN,광양읍
3,1980.0,52.0,127.580683,34.970130,주차장,POINT (127.5806828 34.97013011),NaN,NaN,광양읍
4,1970.0,64.0,127.580707,34.972523,주차장,POINT (127.5807067 34.97252282),NaN,NaN,광양읍
...,...,...,...,...,...,...,...,...,...
786,NaN,NaN,127.612698,34.970095,충전기,POINT (127.6126981 34.97009451),NaN,급속,광양읍
787,NaN,NaN,127.612698,34.970095,충전기,POINT (127.6126981 34.97009451),NaN,완속,광양읍
788,NaN,NaN,127.610457,34.968399,충전기,POINT (127.6104569 34.96839912),NaN,급속,광양읍
789,NaN,NaN,127.610457,34.968399,충전기,POINT (127.6104569 34.96839912),NaN,완속,광양읍


In [189]:
#임시 저장..ㅎㅎ
table.to_csv('table.csv', index=True, encoding='cp949')